### LLM_Chain

使用本地的开源模型，利用ollama来运行llama2

确保ollama服务处于运行状态

In [23]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

接下来我们就可以尝试使用llm调用llama2模型了。我们问它"LangSmith"是什么。
这是训练数据中没有出现过的东西，所有它并不能给出很好的回答

In [24]:
llm.invoke("how can langsmith help with testing?")

'Langsmith is a language model that can assist with testing in several ways:\n\n1. Automated Testing: Langsmith can be used to generate test cases based on the specifications of a software system, allowing for automated testing. This can save time and resources compared to manual testing methods.\n2. Code Review: Langsmith can analyze code written in any programming language and provide feedback on coding standards, best practices, and potential issues. This can help identify areas where testing may be necessary or beneficial.\n3. Test Data Generation: Langsmith can generate test data based on the input parameters of a software system, allowing for more comprehensive and efficient testing.\n4. Regression Testing: Langsmith can assist in regression testing by identifying areas of code that have changed since the last release and suggesting additional test cases to ensure that these changes have not introduced any bugs or issues.\n5. Performance Testing: Langsmith can analyze code for po

我们也可以使用提示模板来指导模型作出响应。提示模版的作用是，将用户的原始输入转换为对于llm更好的输入。

In [25]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

现在我们可以将这些组合成一个简单的LLM链

In [26]:
chain = prompt | llm 

我们现在可以调用它并问同样的问题。虽然它仍然不知道答案，但能以一种更适合技术作家的语气来回应!

In [27]:
chain.invoke({"input": "how can langsmith help with testing?"})

'As a technical documentation writer, I can provide several ways in which LangSmith can help with testing:\n\n1. Testing Language Modules: LangSmith provides a variety of language modules that can be used to test different aspects of your system. For example, you can use the "langsmith-test" module to write unit tests for your system\'s code, or the "langsmith-integration-test" module to test how different components of your system interact with each other.\n2. Automated Testing: LangSmith provides tools for automating testing, such as the "langsmith-automated-testing" module. This can help you save time and effort by generating test cases automatically based on your system\'s code and requirements.\n3. Code Coverage Analysis: LangSmith provides tools for analyzing the coverage of your tests, so you can identify areas of your system that need more testing. For example, the "langsmith-code-coverage" module can generate reports on the percentage of your code that is covered by your tests

ChatModel的输出(也是这个链的输出)是一条消息。然而，使用字符串通常要方便得多。让我们添加一个简单的输出解析器，将聊天消息转换为字符串。

In [28]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

现在可以把这个添加到前面的链中:

In [29]:
chain = prompt | llm | output_parser

现在可以调用它并问同样的问题。现在答案将是一个字符串(而不是ChatMessage)。·

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

### Retrieval_Chain

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires installing BeautifulSoup:

In [3]:
pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


After that, we can import and use WebBaseLoader.

In [4]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

For embedding models, we once again provide examples for accessing via API or by running local models.

Make sure you have Ollama running (same set up as with the LLM).

In [5]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.


First we need to install the required packages for that:

In [6]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


Then we can build our index:

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.



In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:

In [9]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results. This means that langsmith can provide a way for users to view and analyze their test data, such as test scores or results of experiments, in a visual format. This can be helpful for understanding and interpreting test data, identifying trends and patterns, and making informed decisions based on the test results.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [10]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key

In [11]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. Prototyping: LangSmith allows for quick experimentation between prompts, model types, and retrieval strategy, making it easier to understand how the model is performing and debug where it is failing.
2. Initial Test Set: LangSmith enables developers to create datasets of inputs and reference outputs, which can be used to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
3. Comparison View: LangSmith provides a user-friendly comparison view for test runs, allowing developers to track and diagnose regressions in test scores across multiple revisions of their application.
4. Playground: LangSmith offers a playground environment for rapid iteration and experimentation, allowing developers to quickly test out different prompts and models without having to manually run and analyze each one.

Overall, LangSmith can help stre

### Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

1.The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.

2.The final LLM chain should likewise take the whole history into account

#### Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [13]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this out by passing in an instance where the user asks a follow-up question.

In [14]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, weâ€™ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if theyâ€™re just starting their journey.Prototypingâ€‹Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing â€” and debug where it is failing â€” is incredibly important for this phase.Debuggingâ€‹When developin

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [18]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="This allows you to quickly test out different prompts and models. You can open the playground from any prompt or model run in your trace.\nEvery playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testingâ€‹Beta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, itâ€™s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly itâ€™s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedbackâ€‹When launching your application to 

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!

### Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

**NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.**

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1.The retriever we just created. This will let it easily answer questions about LangSmith

2.A search tool. This will let it easily answer questions that require up-to-date information.

First, let's set up a tool for the retriever we just created:

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

The search tool that we will use is Tavily. This will require an API key (they have generous free tier). After creating it on their platform, you need to set it as an environment variable:

In [21]:
export TAVILY_API_KEY=...

SyntaxError: invalid syntax (633340698.py, line 1)

If you do not want to set up an API key, you can skip creating this tool.

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)

We can now create a list of the tools we want to work with:

In [ ]:
tools = [retriever_tool, search]

Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly - for a deeper dive into what exactly is going on, check out the Agent's Getting Started documentation

Install langchain hub first

In [ ]:
pip install langchainhub

Install the langchain-openai package To interact with OpenAI we need to use langchain-openai which connects with OpenAI SDK[https://github.com/langchain-ai/langchain/tree/master/libs/partners/openai].

In [ ]:
pip install langchain-openai

Now we can use it to get a predefined prompt

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `openai_api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:



In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

We can ask it about the weather:

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})

We can have conversations with it:

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})